#Conexión a la base de datos origen y destino

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [ ]:
engine = create_engine('sqlite:///Northwind_large.sqlite')
engine2 = create_engine('sqlite:///DW_Northwind.db')

#Dimensión Customer


In [ ]:
#Obtener la tabla customer
customer_dim = pd.read_sql_table('Customer', con=engine)
#Verificar valores nulos
customer_dim.isnull().sum()
#Reemplzar valores nulos por 0
customer_dim['Fax'].replace(np.nan, "0",  inplace = True)
#Cambio de nombre al identificador
customer_dim.rename(columns={'Id': 'CustomerId'}, inplace=True)
#Carga de datos
customer_dim.to_sql(name='dim_customer', con=engine2, if_exists='append', index=False)

#Dimensión Employee

In [ ]:
#Obtener la tabla employee
employee_dim = pd.read_sql_table('Employee', con=engine)
#Eliminar columnas innecesarias
employee_dim.drop(['Extension', 'Photo', 'Notes', 'ReportsTo', 'PhotoPath'], axis=1, inplace=True)
#Verificar valores nulos
employee_dim.isnull().sum()
#Cambio de nombre al identificador
employee_dim.rename(columns={'Id': 'EmployeeId'}, inplace=True)
#Carga de datos
employee_dim.to_sql(name='dim_employee', con=engine2, if_exists='append', index=False)

#Dimensión Category

In [ ]:
#Obtener la tabla category
category_dim = pd.read_sql_table('Category', con=engine)
#Verificar valores nulos
category_dim.isnull().sum()
#Cambio de nombre al identificador
category_dim.rename(columns={'Id': 'CategoryId'}, inplace=True)
#Carga de datos
category_dim.to_sql(name='dim_category', con=engine2, if_exists='append', index=False)

#Dimensión Product

In [ ]:
#Obtener la tabla Product
product_dim = pd.read_sql_table('Product', con=engine)
#Eliminar columnas innecesarias
product_dim.drop(['SupplierId', 'QuantityPerUnit', 'UnitPrice', 'UnitsInStock', 'UnitsOnOrder', 'ReorderLevel', 'Discontinued'], axis=1, inplace=True)
#Verificar valores nulos
product_dim.isnull().sum()
#Cambio de nombre al identificador
product_dim.rename(columns={'Id': 'ProductId'}, inplace=True)
#Carga de datos
product_dim.to_sql(name='dim_product', con=engine2, if_exists='append', index=False)

#Dimensión Region

In [ ]:
#Obtener la tabla Region
region_dim = pd.read_sql_table('Region', con=engine)
#Verificar valores nulos
region_dim.isnull().sum()
#Cambio de nombre al identificador
region_dim.rename(columns={'Id': 'RegionId'}, inplace=True)
#Carga de datos
region_dim.to_sql(name='dim_region', con=engine2, if_exists='append', index=False)

#Dimensión Supplier

In [ ]:
#Obtener la tabla Supplier
supplier_dim = pd.read_sql_table('Supplier', con=engine)
#Eliminar columnas innecesarias
supplier_dim.drop(['HomePage'], axis=1, inplace=True)
#Verificar valores nulos
supplier_dim.isnull().sum()
#Reemplzar valores nulos por 0
supplier_dim['Fax'].replace(np.nan, "0",  inplace = True)
#Cambio de nombre al identificador
supplier_dim.rename(columns={'Id': 'SupplierId'}, inplace=True)
#Carga de datos
supplier_dim.to_sql(name='dim_supplier', con=engine2, if_exists='append', index=False)

#Dimensión Territory

In [ ]:
#Obtener la tabla Territory
territory_dim = pd.read_sql_table('Territory', con=engine)
#Verificar valores nulos
territory_dim.isnull().sum()
#Cambio de nombre al identificador
territory_dim.rename(columns={'Id': 'TerritoryId'}, inplace=True)
#Carga de datos
territory_dim.to_sql(name='dim_territory', con=engine2, if_exists='append', index=False)

#Dimensión Time

In [ ]:
#Función para crear la dimensión tiempo
def create_time_table(start='2012-01-01', end='2016-12-31'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    df['TimeId'] = df.index + 1
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['month_name'] = df.date.dt.month_name()
    df['day'] = df.date.dt.day
    df['day_name'] = df.date.dt.day_name()
    df['day_week'] = df.date.dt.dayofweek
    df['week'] = df.date.dt.weekofyear
    df['quarter'] = df.date.dt.quarter
    
    df = df[['TimeId', 'date', 'year', 'month', 'month_name', 'day', 'day_name', 'day_week', 'week', 'quarter']] 
    
    return df

In [ ]:
#Obtener la dimensión tiempo
time_dim = create_time_table()
#Carga de datos
time_dim.to_sql(name='dim_time', con=engine2, if_exists='append', index=False)
time_dim

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.


,TimeId,date,year,month,month_name,day,day_name,day_week,week,quarter
0,1,2012-01-01,2012,1,January,1,Sunday,6,52,1
1,2,2012-01-02,2012,1,January,2,Monday,0,1,1
2,3,2012-01-03,2012,1,January,3,Tuesday,1,1,1
3,4,2012-01-04,2012,1,January,4,Wednesday,2,1,1
4,5,2012-01-05,2012,1,January,5,Thursday,3,1,1
...,...,...,...,...,...,...,...,...,...,...
1822,1823,2016-12-27,2016,12,December,27,Tuesday,1,52,4
1823,1824,2016-12-28,2016,12,December,28,Wednesday,2,52,4
1824,1825,2016-12-29,2016,12,December,29,Thursday,3,52,4
1825,1826,2016-12-30,2016,12,December,30,Friday,4,52,4


#Tabla de hechos Order_Fact

In [ ]:
#Obtener las tablas necesarias para la tabla de hechos
Order_tbl = pd.read_sql_table('Order', con=engine)
Order_tbl.rename(columns={'Id': 'OrderId'}, inplace=True)
#Cambio de formato de fechas
Order_tbl['OrderDate']= pd.to_datetime(Order_tbl['OrderDate'],infer_datetime_format=True)
Order_tbl['OrderDate']=pd.to_datetime(Order_tbl['OrderDate']).dt.normalize()
OrderDetail = pd.read_sql_table('OrderDetail', con=engine)
OrderDetail.rename(columns={'Id': 'OrderDetailId'}, inplace=True)
product_tbl = pd.read_sql_table('Product', con=engine)
product_tbl.rename(columns={'Id': 'ProductId'}, inplace=True)
category_tbl = pd.read_sql_table('Category', con=engine)
category_tbl.rename(columns={'Id': 'CategoryId'}, inplace=True)
suppler_tbl = pd.read_sql_table('Supplier', con=engine)
suppler_tbl.rename(columns={'Id': 'SupplierId'}, inplace=True)

In [ ]:
#Proceso de unión de tablas para generar las tabla de hechos
product_tbl = pd.merge(product_tbl, category_tbl[['CategoryId']], on='CategoryId', how='left')
product_tbl = pd.merge(product_tbl, suppler_tbl[['SupplierId']], on='SupplierId', how='left')
OrderDetail = pd.merge(OrderDetail, product_tbl[['ProductId', 'CategoryId', 'SupplierId']], on='ProductId', how='left')
Order_Fact = pd.merge(OrderDetail, Order_tbl[['OrderId', 'CustomerId', 'EmployeeId', 'OrderDate']], on='OrderId', how='left')
Order_Fact = Order_Fact.merge(time_dim[['date', 'TimeId']], left_on='OrderDate', right_on='date')
Order_Fact = Order_Fact[['OrderDetailId', 'CustomerId', 'EmployeeId', 'ProductId', 'SupplierId', 'TimeId','UnitPrice','Quantity']]
Order_Fact.rename(columns={'OrderDetailId': 'OrderFactId'}, inplace=True)
#Carga de datos
Order_Fact.to_sql(name='Order_Fact', con=engine2, if_exists='append', index=False)
Order_Fact


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,OrderFactId,CustomerId,EmployeeId,ProductId,SupplierId,TimeId,UnitPrice,Quantity
0,10248/11,VINET,5,11,5,186,14.00,12
1,10248/42,VINET,5,42,20,186,9.80,10
2,10248/72,VINET,5,72,14,186,34.80,5
3,10249/14,TOMSP,6,14,6,187,18.60,9
4,10249/51,TOMSP,6,51,24,187,42.40,40
...,...,...,...,...,...,...,...,...
621878,24743/32,FOLIG,7,32,14,1106,32.00,4
621879,24743/41,FOLIG,7,41,19,1106,9.65,24
621880,24743/75,FOLIG,7,75,12,1106,7.75,26
621881,24743/77,FOLIG,7,77,12,1106,13.00,5


In [ ]:
#Carga de datos
Order_Fact.to_sql(name='Order_Fact', con=engine2, if_exists='append', index=False)
Order_Fact

,OrderFactId,CustomerId,EmployeeId,ProductId,SupplierId,TimeId,UnitPrice,Quantity
0,10248/11,VINET,5,11,5,186,14.00,12
1,10248/42,VINET,5,42,20,186,9.80,10
2,10248/72,VINET,5,72,14,186,34.80,5
3,10249/14,TOMSP,6,14,6,187,18.60,9
4,10249/51,TOMSP,6,51,24,187,42.40,40
...,...,...,...,...,...,...,...,...
621878,24743/32,FOLIG,7,32,14,1106,32.00,4
621879,24743/41,FOLIG,7,41,19,1106,9.65,24
621880,24743/75,FOLIG,7,75,12,1106,7.75,26
621881,24743/77,FOLIG,7,77,12,1106,13.00,5
